In [9]:
using Pkg: @pkg_str
pkg"activate .."
pkg"add DataStructures"

Activating environment at `~/JuliaEnvs/WrapperSimplification/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/Invenia`
  Updating git-repo `git@gitlab.invenia.ca:invenia/PackageRegistry.git`
 Resolving package versions...
  Updating `~/JuliaEnvs/WrapperSimplification/Project.toml`
  [864edb3b] + DataStructures v0.17.0
  Updating `~/JuliaEnvs/WrapperSimplification/Manifest.toml`
  [864edb3b] + DataStructures v0.17.0
  [bac558e1] + OrderedCollections v1.1.0
  [2a0f44e3] + Base64 
  [8ba89e20] + Distributed 
  [b77e0a4c] + InteractiveUtils 
  [56ddb016] + Logging 
  [d6f4376e] + Markdown 
  [9a3f8284] + Random 
  [9e88b42a] + Serialization 
  [6462fe0b] + Sockets 
  [8dfed614] + Test 


In [22]:
using LinearAlgebra
using DataStructures
using Test

In [1]:
x = [10a + b for a in 1:3, b in 1:3]

3×3 Array{Int64,2}:
 11  12  13
 21  22  23
 31  32  33

In [4]:
Transpose(Transpose(x))

3×3 Transpose{Int64,Transpose{Int64,Array{Int64,2}}}:
 11  12  13
 21  22  23
 31  32  33

In [40]:
# this is not a const, on purpose, we mutate it
# but we always pass it in as an default argument to the function that uses it
# thus causing that function to respecialize when it changes
ALL_RULES = ()  


()

In [41]:
trule1(::Any) = nothing
function trule1(x::Transpose{V, Transpose{V, T}}) where {V, T}
    #(score = 2, type=T, thunk=()::T->parent(parent(x)))
    (state=parent(parent(x)), score=2)
end


ALL_RULES = (ALL_RULES..., trule1)

(trule1,)

In [67]:
function simplify(x, allrules=ALL_RULES; max_steps=10)
    partial_solutions = PriorityQueue{Any, Int}(Base.Order.Reverse)
    function pend!(x, score)
        state = (typeof(x), x)
        if !haskey(partial_solutions, state) 
            enqueue!(partial_solutions, state=>score)
        end
        return nothing
    end
    function next()
        (type, val), score = peek(partial_solutions)
        return val, score
    end
    
    pend!(x, 0)
    for _ in 1:max_steps
        val, score = next()
        for rule in allrules
            res = rule(val)
            res === nothing && continue
            new_val, score_change = res
            new_score = score + score_change
            pend!(new_val, new_score)
        end
    end
    return partial_solutions
end

simplify (generic function with 2 methods)

In [68]:
simplify(x)

PriorityQueue{Any,Int64,Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}} with 1 entry:
  (Array{Int64,2}, [11 12 13; 21 22 23; 31 32 33]) => 0

In [69]:
ttx = Transpose(Transpose(x))
simplify(ttx)

PriorityQueue{Any,Int64,Base.Order.ReverseOrdering{Base.Order.ForwardOrdering}} with 2 entries:
  (Array{Int64,2}, [11 12 13; 21 22 23; 31 32 33])                          => 2
  (Transpose{Int64,Transpose{Int64,Array{Int64,2}}}, [11 12 13; 21 22 23; … => 0

In [71]:
Transpose(Diagonal(x))

3×3 Transpose{Int64,Diagonal{Int64,Array{Int64,1}}}:
 11   0   0
  0  22   0
  0   0  33

In [76]:
Diagonal(Transpose(x))

3×3 Diagonal{Int64,Array{Int64,1}}:
 11   ⋅   ⋅
  ⋅  22   ⋅
  ⋅   ⋅  33